In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
import re
from sqlalchemy import create_engine, text


#Reemplazar end point
pd.options.display.max_rows = 20
DB_USERNAME = ''
DB_PASSWORD = ''
DB_HOST = ''
DB_PORT= ''
DB_NAME = ''
engine=create_engine(f'postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}', max_overflow=20, client_encoding='utf8')

def runQuery(sql):
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

In [87]:
query1="""
select edbc.tmk, edbc.codenc, edbp.competencias_funcionales, edbp.numeros,
edbp.lectura_voz_alta, edbp.fluidez_lectura, edbp.cartografia, edbp.competencias_organizacionales, 
edbp.sumar, edbp.coherencia_entre_numeros, 
edbs.como_conocio_cnc, edbs.fecha_registro, 
edbs.ciudad as "ciudad_encuestador", edbs.experiencia, edbs.tipo,
edbs.barrio, edbs.fecha_nacimiento, edbs.localidad,
edbs.educacion_formal, edbs.estado_educacion, edbs.nombre_educacion
from ecar_dwh.ecar_dwh_base_condec edbc 
left join ecar_dwh.ecar_dwh_base_psicotecnica edbp on edbp.codenc=edbc.codenc 
left join ecar_dwh.ecar_dwh_base_sociodemografica edbs on edbs.codenc=edbp.codenc """

query2="""
select edbc.tmk, edbc.codenc, T.*
from ecar_dwh.ecar_dwh_base_condec edbc 
left join (
select id_bee, codenc, telelink, fecha, duracion, edad, "cod_ciudad", "ciudad", estrato as "cod_estrato", 
edcc.etiqueta as "estrato" 
from (
select id_bee, codenc, telelink, fecha, duracion, edad, ciudad as "cod_ciudad", edcc.etiqueta as "ciudad", estrato 
from ecar_dwh.ecar_dwh_base_encuestas_efectivas edbee 
inner join ecar_dwh.ecar_dwh_correspondencia_codigos edcc on edbee.ciudad=edcc.codigo
where edcc.campo='ciudad') tt
inner join ecar_dwh.ecar_dwh_correspondencia_codigos edcc on tt.estrato=edcc.codigo
where edcc.campo='estrato')T on T.codenc=edbc.codenc
"""

query3="""
select edbc.tmk, edbc.codenc, edbma.calls, edbma.espera, edbma.avg_espera, edbma.hablado, 
edbma.avg_hablado, edbma.disponible, edbma.avg_disponible, edbma.pausas, edbma.avg_pausas,
edbma.muerto, edbma.avg_muerto, edbma.encuesta, edbma.conectado, edbma.archivo
from ecar_dwh.ecar_dwh_base_marcador_automatico edbma
left join ecar_dwh.ecar_dwh_base_condec edbc 
using(codenc) """


query4="""
select *  from ecar_dwh.ecar_dwh_muestra_ECAR_mensual_cuotas """


#df: Códigos left sociodemografica y psicotecnica
df=runQuery(query1)
#df2: Codigos left efectivas 
df2=runQuery(query2)
#df3: ´Marcador automático left códigos
df3=runQuery(query3)
#df4: Cuotas
df4=runQuery(query4)


In [91]:
for var in ['barrio', 'fecha_nacimiento', 'localidad', 'educacion_formal','estado_educacion', 'nombre_educacion']:
    df[var]=(df[var].astype(str)).str.lower()

In [3]:
df3.head()

,tmk,codenc,calls,espera,avg_espera,hablado,avg_hablado,disponible,avg_disponible,pausas,avg_pausas,muerto,avg_muerto,encuesta,conectado,archivo
0,Encuestador 135,121,6085,54:52:52,0.2157,102:18:07,0.4022,23:17:08,0.0915,73:54:41,0.2906,0:56:32,0.0037,101:21:35,1:45:29,VCD_Abril_2018.xlsx
1,Encuestador 266,148,7170,65:00:35,0.2558,118:49:58,0.4676,18:02:06,0.0710,52:16:36,0.2057,1:14:11,0.0049,117:35:47,1:13:53,VCD_Abril_2018.xlsx
2,Encuestador 17,47,2432,22:09:28,0.2456,38:59:31,0.4321,9:46:41,0.1084,19:18:30,0.2140,0:21:51,0.0040,38:37:40,0:24:00,VCD_Abril_2018.xlsx
3,Encuestador 82,170,432,5:45:52,0.2859,7:54:21,0.3921,1:22:57,0.0686,5:06:41,0.2535,0:04:36,0.0038,7:49:45,1:00:25,VCD_Abril_2018.xlsx
4,Encuestador 116,117,7829,72:37:55,0.2880,105:02:58,0.4166,27:18:28,0.1083,47:11:00,0.1871,1:27:06,0.0058,103:35:52,0:47:57,VCD_Abril_2018.xlsx


In [4]:
df3['archivo']=df3['archivo'].astype(str)
df3['mes']=df3['archivo'].str.split('_',0).str[1]
df3['año']=df3['archivo'].str.split('_',0).str[2].str.split('.',0).str[0]
df3.drop('archivo', axis=1, inplace=True)
for cols in ['espera', 'hablado', 'disponible', 'pausas', 'muerto', 'encuesta', 'conectado']:
    df3[cols]=pd.to_timedelta(df3[cols]).dt.total_seconds()

In [5]:
print(df3.shape)
df3.head()

(1202, 17)


,tmk,codenc,calls,espera,avg_espera,hablado,avg_hablado,disponible,avg_disponible,pausas,avg_pausas,muerto,avg_muerto,encuesta,conectado,mes,año
0,Encuestador 135,121,6085,197572.0,0.2157,368287.0,0.4022,83828.0,0.0915,266081.0,0.2906,3392.0,0.0037,364895.0,6329.0,Abril,2018
1,Encuestador 266,148,7170,234035.0,0.2558,427798.0,0.4676,64926.0,0.0710,188196.0,0.2057,4451.0,0.0049,423347.0,4433.0,Abril,2018
2,Encuestador 17,47,2432,79768.0,0.2456,140371.0,0.4321,35201.0,0.1084,69510.0,0.2140,1311.0,0.0040,139060.0,1440.0,Abril,2018
3,Encuestador 82,170,432,20752.0,0.2859,28461.0,0.3921,4977.0,0.0686,18401.0,0.2535,276.0,0.0038,28185.0,3625.0,Abril,2018
4,Encuestador 116,117,7829,261475.0,0.2880,378178.0,0.4166,98308.0,0.1083,169860.0,0.1871,5226.0,0.0058,372952.0,2877.0,Abril,2018


In [48]:
#Eliminando duplicados 
VCD = df3.loc[:, df3.columns != 'codenc'].drop_duplicates()
VCD['n_mes'] = VCD['mes']
VCD.shape

(1185, 17)

In [49]:
VCD['n_mes'] = VCD['n_mes'].replace({'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8, 
                     'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12})
VCD['dia'] = 1

In [51]:
VCD['fecha'] = VCD['año'].astype(str)+ ' ' +VCD['n_mes'].astype(str)+ ' ' +VCD['dia'].astype(str)

In [53]:
VCD['fecha'] = pd.to_datetime(VCD['fecha']) 

In [54]:
VCD.head()

,tmk,calls,espera,avg_espera,hablado,avg_hablado,disponible,avg_disponible,pausas,avg_pausas,muerto,avg_muerto,encuesta,conectado,mes,año,n_mes,dia,fecha
0,Encuestador 135,6085,197572.0,0.2157,368287.0,0.4022,83828.0,0.0915,266081.0,0.2906,3392.0,0.0037,364895.0,6329.0,Abril,2018,4,1,2018-04-01
1,Encuestador 266,7170,234035.0,0.2558,427798.0,0.4676,64926.0,0.0710,188196.0,0.2057,4451.0,0.0049,423347.0,4433.0,Abril,2018,4,1,2018-04-01
2,Encuestador 17,2432,79768.0,0.2456,140371.0,0.4321,35201.0,0.1084,69510.0,0.2140,1311.0,0.0040,139060.0,1440.0,Abril,2018,4,1,2018-04-01
3,Encuestador 82,432,20752.0,0.2859,28461.0,0.3921,4977.0,0.0686,18401.0,0.2535,276.0,0.0038,28185.0,3625.0,Abril,2018,4,1,2018-04-01
4,Encuestador 116,7829,261475.0,0.2880,378178.0,0.4166,98308.0,0.1083,169860.0,0.1871,5226.0,0.0058,372952.0,2877.0,Abril,2018,4,1,2018-04-01


In [75]:
#Agrupando y sumando datos con más de un TMK por FECHA

VCD_id = VCD.groupby(['tmk','fecha']).agg({'calls': ['count', 'sum'], 'espera': ['sum'], 'hablado': ['sum'], 'disponible': ['sum'], 
                                           'pausas': ['sum'], 'muerto': ['sum'], 'encuesta': ['sum']}).reset_index() #, 'conectado': ['sum']

VCD_id.columns=['tmk', 'fecha', 'N', 'llamadas', 'espera', 'hablado', 'disponible', 'pausas', 'muerto', 'cliente']

#Recalulando los porcentajes
VCD_id['total'] = VCD_id['espera'] + VCD_id['hablado'] + VCD_id['disponible'] + VCD_id['pausas']
VCD_id['espera_porct'] = (VCD_id['espera'] / VCD_id['total'] )*100
VCD_id['hablado_porct'] = (VCD_id['hablado'] / VCD_id['total'] )*100
VCD_id['disponible_porct'] = (VCD_id['disponible'] / VCD_id['total'])*100 
VCD_id['pausas_porct'] = (VCD_id['pausas'] / VCD_id['total'] )*100

print(VCD_id.shape)
VCD_id.head()

#La base con registro únicos es VCD_id

(1181, 15)


,tmk,fecha,N,llamadas,espera,hablado,disponible,pausas,muerto,cliente,total,espera_porct,hablado_porct,disponible_porct,pausas_porct
0,Encuestador 1,2017-07-01,1,4429,150572.0,313690.0,42598.0,116624.0,2193.0,311497.0,623484.0,24.150098,50.312438,6.832252,18.705211
1,Encuestador 10,2018-04-01,1,647,18555.0,31013.0,9059.0,14190.0,171.0,30842.0,72817.0,25.481687,42.590329,12.440776,19.487208
2,Encuestador 10,2018-05-01,1,2369,63164.0,91214.0,20844.0,47569.0,407.0,90807.0,222791.0,28.351235,40.941510,9.355854,21.351401
3,Encuestador 100,2017-07-01,1,6077,218264.0,352053.0,102520.0,145907.0,2567.0,349486.0,818744.0,26.658394,42.999155,12.521618,17.820833
4,Encuestador 100,2017-08-01,1,5572,216808.0,340585.0,109026.0,139608.0,4221.0,336364.0,806027.0,26.898355,42.254788,13.526346,17.320512
